In [24]:
import pandas as pd
import numpy as np
import os
from pyspark.sql.functions import *

# Подключение к кластеру

In [2]:
from pyspark.sql.types import *
from pyspark.sql import SparkSession
import pyspark
# Create a SparkSession
spark = (SparkSession
    .builder
    .appName("Lab2")
    .getOrCreate())



23/10/12 14:19:58 WARN Utils: Your hostname, dortp58-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/10/12 14:19:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/12 14:19:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Чтение и обработка данных

In [3]:
df = pd.read_csv("book1-100k.csv")
schema = df.dtypes.to_dict()
schema


{'Id': dtype('int64'),
 'Name': dtype('O'),
 'RatingDist1': dtype('O'),
 'pagesNumber': dtype('int64'),
 'RatingDist4': dtype('O'),
 'RatingDistTotal': dtype('O'),
 'PublishMonth': dtype('int64'),
 'PublishDay': dtype('int64'),
 'Publisher': dtype('O'),
 'CountsOfReview': dtype('int64'),
 'PublishYear': dtype('int64'),
 'Language': dtype('O'),
 'Authors': dtype('O'),
 'Rating': dtype('float64'),
 'RatingDist2': dtype('O'),
 'RatingDist5': dtype('O'),
 'ISBN': dtype('O'),
 'RatingDist3': dtype('O')}

In [4]:
book_schema = StructType([
    StructField("Id", IntegerType()),
    StructField("Name", StringType()),
    StructField("PagesNumber", IntegerType()),
    StructField("Authors", StringType()),
    StructField("Publisher", StringType()),
    StructField("Language", StringType()),
    StructField("Rating", DoubleType()),
    StructField("RatingDist1", StringType()),
    StructField("RatingDist2", StringType()),
    StructField("RatingDist3", StringType()),
    StructField("RatingDist4", StringType()),
    StructField("RatingDist5", StringType()),
    StructField("RatingDistTotal", StringType()),
    StructField("CountsOfReview", IntegerType()),
    StructField("PublishDay", IntegerType()),
    StructField("PublishMonth", IntegerType()),
    StructField("PublishYear", IntegerType()),
    StructField("ISBN", StringType())
])

In [5]:
cols = [i.name for i in book_schema.fields]

In [7]:
import glob
all_files = glob.glob(r'./' + "/b*.csv")
books = spark.createDataFrame([], schema=book_schema)
for filename in all_files:
    df = pd.read_csv(filename)
    if 'PagesNumber' not in df.columns:
        df = df.rename(columns={"pagesNumber": "PagesNumber"})
    spak_df = spark.createDataFrame(df[cols], schema=book_schema)
    books = books.union(spak_df)
    print(filename)

./book400k-500k.csv
./book900k-1000k.csv
./book1400k-1500k.csv
./book600k-700k.csv
./book1200k-1300k.csv
./book1700k-1800k.csv
./book1900k-2000k.csv
./book700k-800k.csv
./book3000k-4000k.csv
./book1500k-1600k.csv
./book4000k-5000k.csv
./book1000k-1100k.csv
./book1800k-1900k.csv
./book1600k-1700k.csv
./book1300k-1400k.csv
./book500k-600k.csv
./book800k-900k.csv
./book300k-400k.csv
./book200k-300k.csv
./book1100k-1200k.csv
./book100k-200k.csv
./book2000k-3000k.csv
./book1-100k.csv


Читаем файлы user_rating

In [8]:
df = pd.read_csv("user_rating_0_to_1000.csv")
schema = df.dtypes.to_dict()
schema

{'ID': dtype('int64'), 'Name': dtype('O'), 'Rating': dtype('O')}

In [9]:
user_schema = StructType([
    StructField("ID", IntegerType()),
    StructField("Name", StringType()),
    StructField("Rating", StringType())
])

In [10]:
cols_u = ["ID", "Name", "Rating"]

In [11]:
import glob
all_files = glob.glob(r'./' + "/u*.csv")
users = spark.createDataFrame([], schema=user_schema)
for filename in all_files:
    df = pd.read_csv(filename)
    spak_df = spark.createDataFrame(df[cols_u], schema=user_schema)
    users = users.union(spak_df)
    print(filename)

./user_rating_4000_to_5000.csv
./user_rating_2000_to_3000.csv
./user_rating_3000_to_4000.csv
./user_rating_6000_to_11000.csv
./user_rating_1000_to_2000.csv
./user_rating_0_to_1000.csv
./user_rating_5000_to_6000.csv


Преобразование в parquet и запись

На hdfs никак не получается

In [129]:
books.write.parquet("hdfs://localhost:9870/user/books.parquet")

Py4JJavaError: An error occurred while calling o813.parquet.
: org.apache.hadoop.ipc.RpcException: RPC response exceeds maximum data length
	at org.apache.hadoop.ipc.Client$IpcStreams.readResponse(Client.java:1936)
	at org.apache.hadoop.ipc.Client$Connection.receiveRpcResponse(Client.java:1238)
	at org.apache.hadoop.ipc.Client$Connection.run(Client.java:1134)


In [132]:
books.write.parquet("books.parquet",mode = "overwrite")
users.write.parquet("users.parquet",mode = "overwrite")
books.write.csv("books.csv",mode = "overwrite")
users.write.csv("users.csv",mode = "overwrite")

23/10/12 00:33:35 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
23/10/12 00:33:53 WARN TaskSetManager: Stage 9 contains a task of very large size (2954 KiB). The maximum recommended task size is 1000 KiB.
23/10/12 00:33:57 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
23/10/12 00:33:57 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 84,44% for 9 writers
23/10/12 00:33:57 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
23/10/12 00:33:57 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 84,44% for 9 writers
23/10/12 00:33:57 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720

# Измерение скорости чтения

Для датасета книг

In [136]:
import time
start = time.time()
books_parquet = spark.read.parquet("books.parquet")
end_p = time.time()-start

start = time.time()
books_csv = spark.read.csv("books.csv")
end_csv = time.time()-start

print("Время чтения parquet: ", end_p)
print("Время чтения сsv: ", end_csv)

Время чтения parquet:  0.289625883102417
Время чтения сsv:  2.851456880569458


Для датасета юзер скоров

In [137]:
import time
start = time.time()
users_parquet = spark.read.parquet("users.parquet")
end_p = time.time()-start

start = time.time()
users_csv = spark.read.csv("users.csv")
end_csv = time.time()-start

print("Время чтения parquet: ", end_p)
print("Время чтения сsv: ", end_csv)

Время чтения parquet:  0.18186259269714355
Время чтения сsv:  0.39438915252685547


Топ 10 книг с наибольшим числом ревью

In [18]:
books.sort(books.CountsOfReview.desc()).select("Name").show(10)

23/10/12 14:36:58 WARN TaskSetManager: Stage 5 contains a task of very large size (2954 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+
|                Name|
+--------------------+
|The Hunger Games ...|
|Twilight (Twiligh...|
|      The Book Thief|
|            The Help|
|Harry Potter and ...|
|The Giver (The Gi...|
| Water for Elephants|
|The Girl with the...|
|Harry Potter and ...|
|The Lightning Thi...|
+--------------------+
only showing top 10 rows



Топ 10 издателей с наибольшим средним числом страниц

In [25]:
books.groupBy("Publisher").avg("PagesNumber").sort(avg("PagesNumber").desc()).show(10)

23/10/12 14:52:59 WARN TaskSetManager: Stage 9 contains a task of very large size (2954 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+------------------+
|           Publisher|  avg(PagesNumber)|
+--------------------+------------------+
|Crafty Secrets Pu...|         1807321.6|
|    Sacred-texts.com|          500000.0|
|Department of Rus...| 322128.5714285714|
|Logos Research Sy...|          100000.0|
|Encyclopedia Brit...|           32642.0|
|Progressive Manag...|        19106.3625|
|Still Waters Revi...|10080.142857142857|
|P. Shalom Publica...|            8539.0|
|Hendrickson Publi...|            6448.0|
|            IEEE/EMB|            6000.0|
+--------------------+------------------+
only showing top 10 rows



10 наиболее активных по число изданных книг лет

In [28]:
books.groupBy("PublishYear").count().sort(count('*').desc()).show(10)

23/10/12 14:58:44 WARN TaskSetManager: Stage 12 contains a task of very large size (2954 KiB). The maximum recommended task size is 1000 KiB.


+-----------+------+
|PublishYear| count|
+-----------+------+
|       2007|129507|
|       2006|122374|
|       2005|117639|
|       2004|105733|
|       2003|104345|
|       2002| 95537|
|       2001| 88228|
|       2000| 87290|
|       2008| 80265|
|       1999| 80155|
+-----------+------+
only showing top 10 rows



# Spark Streaming

In [51]:
from itertools import chain
rating_map = {
    "it was amazing" : 5,
    "really liked it" : 4,
    "liked it" : 3,
    "it was ok" : 2,
    "did not like it" : 1
}

mapping_expr = create_map([lit(x) for x in chain(*rating_map.items())])

In [52]:
mapping_expr

Column<'map(it was amazing, 5, really liked it, 4, liked it, 3, it was ok, 2, did not like it, 1)'>

In [54]:
stream_df = spark.readStream\
    .schema(user_schema)\
    .parquet("./users.parquet")\
    .filter(col("Rating") != "This user doesn't have any rating")\
    .withColumn("RatingValue", mapping_expr[col('Rating')])\
    .withColumn("timestamp", current_timestamp())\
    .withWatermark("timestamp", "10 seconds") \
    .groupBy("Name", window(col("timestamp"), "15 seconds"))\
    .agg(avg("RatingValue").alias("AverageRating"))

In [ ]:
stream_df.writeStream\
    .format("parquet")\
    .option("header", True)\
    .option("path", "./book_ratings.parquet")\
    .option("checkpointLocation", "./checkpoints")\
    .start()\
    .awaitTermination()

23/10/12 17:00:15 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
